In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, LstmMle_LatentSpace, AnalysisLayer
from models_mse import LstmMse
from models_mle import LstmMle_1, LstmMle_2, LstmMle_3
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle, PredictorMseLatentSpaceAnalyser, PredictorMleLatentSpaceAnalyser, PredictorMleSpecial

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters phm data

In [25]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/01_M02_DC_prediction_3.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/phm_data_M02Fold_xx_InputSize12_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 12,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : [2,20], # Individual Threshold for each Sensor
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/phm_M02_3.csv",
    }
}

## Parameters artifical data

In [14]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MSE_model/artifical_data_Fold_xx_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : [0.75,10], # Individual Threshold for each Sensor
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MSE/artifical_2_signals.csv",
    }
}

## Parameters cpps data

In [2]:
param = {
    "data" : {
        "path" : '../../data/cpps_degradation/9/test/cpps_data_degradation_test_09.csv',
        "droped_feature" : [
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/degeneration_cpps_data_09_Fold_xx_InputSize10_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 10,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/cpps_degradation_09.csv", 
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from phm Dataset M02
droped features="stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "Tool"

In [26]:
mean_training_data =[ 0.00048908,  0.02424968,  0.03746501,  0.02320936,  0.04798253,  0.16690744, 0.00635931,  0.11491659,  0.16438863, -0.1371422, -0.1224534, -0.03445835]
var_training_data =[4.71816879e-04, 9.91867997e-01, 1.01665728e+00, 9.90095853e-01, 1.09008254e+00, 5.49661581e-01, 2.40779532e-01, 8.64385070e-01, 5.58986809e-01, 5.68935150e-01, 1.05338930e-10, 9.83200120e-01]

### Mean and Variance from artifical Dataset

In [15]:
mean_training_data= [-0.00526595, -0.00968424]
var_training_data = [49.30277603, 0.4232726 ]

### Mean and Variance form cpps Dataset with degradation (0.000009)

In [3]:
mean_training_data = [0.0543732, 0.03136465, 0.04093888, -0.01314792, 0.018266, 0.05595022, 0.07577236, 0.04436076, 0.06319417, 0.02403438]
var_training_data = [0.105145, 0.06034442, 0.05224029, 0.13971766, 0.07913361, 0.09779795, 0.05276312, 0.06371347, 0.05103405, 0.04241744]

## Create DataLoader

In [16]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(12000, 3)


In [17]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 3])
Size of target data: torch.Size([50, 3])
Data of batch: 1
Size of input data: torch.Size([50, 100, 3])
Size of target data: torch.Size([50, 3])


## Define Model and load Parameters of trained model
### Model for MSE 

In [18]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=param['model']['n_hidden_lstm'], 
                n_hidden_fc=param['model']['n_hidden_fc_1'], 
                n_layers=param['model']['lstm_layer'], 
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc']
                )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

### Model for MLE

In [18]:
model = LstmMle_1(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc=param['model']['n_hidden_fc_1'], 
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [ ]:
model = LstmMle_3(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc_1=param['model']['n_hidden_fc_1'], 
                 n_hidden_fc_2=param['model']['n_hidden_fc_2'],
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

## Initialize Predictor
### Predictor for MSE Model

In [19]:
predictor = PredictorMse(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                         )

### Predictor for MLE Model

In [19]:
predictor = PredictorMle(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                         no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                         )

In [14]:
predictor = PredictorMleSpecial(model=model,
                                path_data=param["data"]["path"],
                                columns_to_ignore=param["data"]["droped_feature"],
                                threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                                no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"],
                                deep=False, 
                                seperate=False,
                                )

## Predict

In [20]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
End of prediction.


## Tag anomalous samples

In [21]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

In [22]:
results.head()

,ID,sine_signal target,sawtooth_signal target,sine_signal predicted,sawtooth_signal predicted,sine_signal reconstruction error,sawtooth_signal reconstruction error,Anomaly Sensor_1,Anomaly Sensor_2
0,100.0,0.094079,-1.559943,0.020426,-0.191629,0.005425,1.872283,0,0
1,101.0,0.213996,-1.146374,0.202862,-1.123945,0.000124,0.000503,0,0
2,102.0,0.350721,-1.672505,0.565588,-1.272157,0.046168,0.160278,0,0
3,103.0,0.564450,-0.276615,0.688174,-1.050457,0.015308,0.598832,0,0
4,104.0,0.692688,-0.955894,0.738744,-1.065945,0.002121,0.012111,0,0


## Combine prediction data with data which was not consider for inference

In [23]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [24]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)